
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# The next 8 lines added to be compatible with version changes (2022/10/13).
# change cuda to 11.1
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.1/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.1/lib64:{ld}"
# change pytorch to 1.9.0 compiled with cuda 11.1
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html


# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

In [43]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/MyDrive/CMPT_CV_lab3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### Data Loader

In [44]:
from detectron2.data.common import data
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''

def get_detection_data(set_name):
  data_dirs = '{}/data'.format(BASE_DIR)
  
  json_file = os.path.join(data_dirs, "train.json")
  with open(json_file) as f:
    imgs_anns = json.load(f)

  objs = []
  dataset = []

  if set_name == "test":
    idx = 1
    record = {}
    for filename in os.listdir(os.path.join(data_dirs, set_name)):
      filename = os.path.join(set_name, filename)
      filename = os.path.join(data_dirs, filename)
      height, width = cv2.imread(filename).shape[:2]
      
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width
      record["annotations"] = []

      dataset.append(record)
      record = {}
      idx += 1

  elif set_name == "train":

    idx = 0
    record = {}
    pre_id = imgs_anns[0]["image_id"]

    for v in imgs_anns:
      obj = {
          "bbox": v["bbox"],
          "bbox_mode": BoxMode.XYWH_ABS,
          "segmentation":v["segmentation"],
          "category_id": 0,
      }
      if v["image_id"] == pre_id:
        objs.append(obj)
      else:
        filename = os.path.join("train", imgs_anns[idx-1]["file_name"])
        filename = os.path.join(data_dirs, filename)
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = pre_id
        record["height"] = height
        record["width"] = width
        record["annotations"] = objs
        dataset.append(record)

        objs = []
        record = {}
        objs.append(obj)
      
      pre_id = v["image_id"]
      idx += 1

    record["annotations"] = objs
    filename = os.path.join("train", imgs_anns[-1]["file_name"])
    filename = os.path.join(data_dirs, filename)
    height, width = cv2.imread(filename).shape[:2]
    record["file_name"] = filename
    record["image_id"] = pre_id
    record["height"] = height
    record["width"] = width
    dataset.append(record)

  return dataset

In [8]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
for d in ["train", "test", "val"]:
  DatasetCatalog.register("plane_" + d, lambda d=d: get_detection_data(d))
  MetadataCatalog.get("plane_" + d).set(thing_classes=["plane"])
plane_matadata = MetadataCatalog.get("plane_train")


In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''
dataset = get_detection_data("train")
for d in random.sample(dataset, 3):
  img = cv2.imread(d["file_name"])
  v = Visualizer(img[:, :, ::-1], metadata=plane_matadata, scale = 0.5)
  out = v.draw_dataset_dict(d)
  cv2_imshow(out.get_image()[:, :, ::-1])

### Set Configs

In [45]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("plane_train",)
cfg.DATASETS.TEST = ("plane_test",)
cfg.DATALOADER.NUM_WORKERS = 2 
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

### Training

In [46]:
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
import copy
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_test_loader, build_detection_train_loader

def custom_mapper(dataset_dict):
  dataset_dict = copy.deepcopy(dataset_dict) # it will be modified by code below
  image = utils.read_image(dataset_dict["file_name"], format = "BGR")
  transform_list = [T.Resize((800, 800)),
                    T.RandomBrightness(0.8, 1.8),
                    T.RandomContrast(0.6, 1.3),
                    T.RandomSaturation(0.8, 1.4),
                    T.RandomRotation(angle=[90, 90]),
                    T.RandomLighting(0.7),
                    T.RandomFlip(prob=0.4, horizontal=False, vertical=True),]
  image, transforms = T.apply_transform_gens(transform_list, image)
  dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))
  annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
  instances = utils.annotations_to_instances(annos, image.shape[:2])
  dataset_dict["instances"] = utils.filter_empty_instances(instances)
  return dataset_dict

class CustomTrainer(DefaultTrainer):
  @classmethod
  def build_train_loader(cls, cfg):
     return build_detection_train_loader(cfg, mapper=custom_mapper)

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [35]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
dataset = get_detection_data("test")
for d in random.sample(dataset, 3):
  img = cv2.imread(d["file_name"])
  outputs = predictor(img)

  v = Visualizer(img[:, :, ::-1], metadata=plane_matadata, scale = 0.5)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Qualitative comparison
'''
dataset = get_detection_data("test")
d = dataset[1]
img = cv2.imread(d["file_name"])
outputs = predictor(img)
v = Visualizer(img[:, :, ::-1], metadata=plane_matadata, scale = 0.5)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
evaluator = COCOEvaluator("plane_train", cfg, False, output_dir= "./output")
val_loader = build_detection_test_loader(cfg, "plane_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))


In [ ]:
import matplotlib.pyplot as plt
OUTPUT_DIR = "{}/output/".format(BASE_DIR)

def json_loader(path):
  res = []
  with open(path, 'r') as f:
    for i in f:
      res.append(json.loads(i))
    return res
p = os.path.join(OUTPUT_DIR, 'metrics.json')
metrics = json_loader(p)

xlist = []
ylist = []
zlist = []
for i in metrics:
  xlist.append((i['iteration']))
  ylist.append((i['fast_rcnn/cls_accuracy']))
  zlist.append(i['total_loss'])

plt.plot(xlist, zlist)
plt.title('training loss')
plt.xlabel('iteration')
plt.ylabel('loss')

### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''


## Part 2: Semantic Segmentation

### Data Loader

In [47]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
from detectron2.utils.visualizer import GenericMask

def get_instance_sample(data, idx, img=None):
  data_anno = data['annotations'][idx]
  img = cv2.imread(data['file_name'])
  x = int(data_anno['bbox'][0])
  y = int(data_anno['bbox'][1])
  w = int(data_anno['bbox'][2])
  h = int(data_anno['bbox'][3])
  obj_img = img[y:y+h, x:x+w]
  gen = GenericMask(data_anno['segmentation'], data['height'], data['width'])
  obj_mask = gen.polygons_to_mask(data_anno['segmentation'])
  obj_mask = obj_mask[y:y+h, x:x+w]

  return obj_img, obj_mask

In [48]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
    # return 10
      return len(self.instance_map)
      

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]
    
    img, mask = get_instance_sample(data, idx[1])
    img = cv2.resize(img, dsize=(128, 128))
    mask = cv2.resize(mask, dsize=(128, 128))
    img, mask = self.numpy_to_tensor(img, mask)

    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("plane_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [49]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # Encoder
        
        self.input_conv = conv(3, 32)
        self.down1 = down(32, 64)
        self.down2 = down(64, 128)
        self.down3 = down(128, 256)
        self.down4 = down(256, 512)

        
        # Decoder
        
        self.up1 = up(512, 256)
        self.up2 = up(256, 128)
        self.up3 = up(128, 64)
        self.up4 = up(64,4)
        self.output_conv = conv(4, 1, False) # ReLu activation is removed to keep the logits for the loss function
        

    def forward(self, input):
      y = self.input_conv(input)
      y = self.down1(y)
      y = self.down2(y)
      y = self.down3(y)
      y = self.down4(y)

      y = self.up1(y)
      y = self.up2(y)
      y = self.up3(y)
      y = self.up4(y)
      output = self.output_conv(y)
      return output

### Training

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''

# Set the hyperparameters
num_epochs = 40
batch_size = 20
learning_rate = 0.01
weight_decay = 0.0005

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
#optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)

# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
    pred = model(img)
    mask = mask.unsqueeze(1) # debug
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


In [ ]:
import matplotlib.pyplot as plt
xlist = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39]
ylist = [0.24261195957660675,0.18665234744548798,0.17510886490345,0.16742222011089325,0.16078521311283112,
         0.1561974436044693,0.1505148559808731,0.14646698534488678,0.1427587866783142,0.13881759345531464,
         0.1355389803647995,0.13121545314788818,0.1279481053352356,0.12490034103393555,0.12202390283346176,
         0.12003765255212784,0.11627399176359177,0.11358490586280823,0.11165279150009155,0.10976707935333252,
         0.10719779878854752,0.10431127995252609,0.10225629806518555,0.10096766799688339,0.09887057542800903,
         0.09771589189767838,0.09607689827680588,0.09449459612369537,0.0941876769065857,0.09143896400928497,
         0.08962993323802948,0.08977644890546799,0.08815287053585052,0.0857917070388794,0.08601415902376175,
         0.08436531573534012,0.08364453911781311,0.08330414444208145,0.08164032548666,0.0808963030576706]

list1 = []
list2 = []

for i in xlist:
  list1.append(i)

for i in ylist:
  list2.append(i)

plt.plot(list1, list2)
plt.title('training loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:

# show the model architecture
from torchsummary import summary
model = MyModel().cuda()
summary(model, (3, 128, 128))

### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output2/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)

total_iou = 0
image_num = 0

for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cpu().detach()
    mask = torch.unsqueeze(mask,1)
    pred = model(img).cpu().detach()

    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
  for i in range(img.shape[0]):
    pi = pred[i]
    pi = nn.Sigmoid()(pi)[0]
    pi = np.array(pi)
    pi = np.rint(pi)

    m = mask[i]
    gt = nn.Sigmoid()(m)[0]
    gt = np.array(gt)
    gt = np.rint(gt)
    
    union = np.sum(np.logical_or(gt, pi))
    intersection= np.sum(np.logical_and(gt, pi))

    iou = (intersection / union)
    image_num += 1
    total_iou += iou

mean = total_iou/image_num

print("\n #images: {}, Mean IoU: {}".format(image_num, mean))


In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''
loader, dataset = get_plane_dataset('train', 3)

img, mask = next(iter(loader))
img = img.cuda()
pred = model(img).cpu().detach()

for i in range(3):
  im = img[i].cpu()
  im = transforms.ToPILImage()(im)
  cv2_imshow(np.array(im))

  p = np.rint(np.array(nn.Sigmoid()(pred[i])[0]))*255
  cv2_imshow(p)

## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [33]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''
def sig(x):
  res = 1 / (1 + np.exp(-x))
  return res

def get_prediction_mask(data):
  
  img = cv2.imread(data['file_name'])
  annotations = data['annotations']
  lenAnnos = len(annotations)
  height = data['height']
  width = data['width']

  temp_pred_mask = np.zeros([height, width])
  temp_gt_mask = np.zeros([height, width])

  predictor_output = predictor(img)
  boxes = predictor_output['instances']
  lenBoxes = len(boxes)



  if lenAnnos == 0:
    for i in range(lenBoxes):
      box = list(boxes[i].get_fields()['pred_boxes'])
      box = np.floor(box[0].cpu().numpy())
      box = np.array(box, dtype=np.uint32)

      x1 = int(box[0])
      y1 = int(box[1])
      x2 = int(box[2])
      y2 = int(box[3])
      
      temp_img = img[y1:y2, x1:x2]
      temp_img = cv2.resize(temp_img, (128, 128), interpolation=cv2.INTER_AREA)
      temp_img = torch.tensor(transforms.ToTensor()(temp_img), device=torch.device('cuda'))
      temp_img = torch.unsqueeze(temp_img, 0)

      temp_img = model(temp_img).cpu().detach().numpy()[0]
      temp_img = cv2.resize(temp_img[0], (x2-x1, y2-y1), interpolation=cv2.INTER_AREA)
      temp_img = np.rint(np.array(sig(temp_img))) * (i + 1)

      mask = temp_pred_mask[y1:y2, x1:x2]
      mask[mask == 0] = 10000

      overlap = np.minimum(mask, temp_img)
      overlap[overlap == 1000] = 0
      temp_pred_mask[y1:y2, x1:x2] = overlap

    for i, j in enumerate(annotations):
      box = j['bbox']
      x1 = int(box[0])
      y1 = int(box[1])
      x2 = int(x1 + box[2])
      y2 = int(y1 + box[3])

      real_mask = detectron2.utils.visualizer.GenericMask(j['segmentation'], height, width).mask
      tmcovering = np.maximum(temp_gt_mask[y1:y2, x1:x2], real_mask[y1:y2, x1:x2] * (i+1))
      temp_gt_mask[y1:y2, x1:x2] = tmcovering

  else:
    for i, j in enumerate(annotations):
      box = j['bbox']
      x1 = int(box[0])
      y1 = int(box[1])
      x2 = int(x1 + box[2])
      y2 = int(y1 + box[3])

      temp_img = img[y1:y2, x1:x2]
      temp_img = cv2.resize(temp_img, (128, 128), interpolation=cv2.INTER_AREA)
      temp_img = torch.tensor(transforms.ToTensor()(temp_img), device=torch.device('cuda'))
      temp_img = torch.unsqueeze(temp_img, 0)


      temp_img = model(temp_img).cpu().detach().numpy()[0]
      temp_img = cv2.resize(temp_img[0], (x2-x1, y2-y1), interpolation=cv2.INTER_AREA)
      temp_img = np.rint(np.array(sig(temp_img))) * (i + 1)

      mask = temp_pred_mask[y1:y2, x1:x2]
      mask[mask == 0] = 10000

      overlap = np.minimum(mask, temp_img)
      overlap[overlap == 1000] = 0
      temp_pred_mask[y1:y2, x1:x2] = overlap

      real_mask = detectron2.utils.visualizer.GenericMask(j['segmentation'], height, width).mask
      tmcovering = np.maximum(temp_gt_mask[y1:y2, x1:x2], real_mask[y1:y2, x1:x2] * (i+1))
      temp_gt_mask[y1:y2, x1:x2] = tmcovering

  gt_mask = torch.tensor(temp_gt_mask, device=torch.device('cuda'))
  pred_mask = torch.tensor(temp_pred_mask, device=torch.device('cuda'))

  return img, gt_mask, pred_mask # gt_mask could be all zero when the ground truth is not given.


### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''

dataset = get_detection_data('test')
r = np.random.randint(0, 50, 3)
for i in r:
  img, gt_mask, pred_mask = get_prediction_mask(dataset[i])

  pred_mask = pred_mask.cpu().numpy()
  max_pred = 255./ (pred_mask.max())
  pred_mask *= max_pred

  cv2_imshow(cv2.resize(img, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_AREA))
  cv2_imshow(cv2.resize(pred_mask, (pred_mask.shape[1], pred_mask.shape[0]), interpolation=cv2.INTER_AREA))

In [38]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
my_data_list = DatasetCatalog.get("plane_{}".format('train'))
#my_data_list = DatasetCatalog.get("plane_{}".format(set_name))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

'''
# Writing the predictions of the test set
'''

my_data_list = DatasetCatalog.get("plane_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred1.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [42]:
def get_detection_data(set_name):
  data_dirs = '{}/data'.format(BASE_DIR)
  
  json_file = os.path.join(data_dirs, "train.json")
  with open(json_file) as f:
    imgs_anns = json.load(f)

  objs = []
  dataset = []

  if set_name == "test":
    idx = 1
    record = {}
    for filename in os.listdir(os.path.join(data_dirs, set_name)):
      filename = os.path.join(set_name, filename)
      filename = os.path.join(data_dirs, filename)
      height, width = cv2.imread(filename).shape[:2]
      
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width
      record["annotations"] = objs

      dataset.append(record)
      record = {}
      idx += 1

  elif set_name == "train":
    idx = 0
    record = {}
    pre_id = imgs_anns[0]["image_id"]

    for v in imgs_anns:
      obj = {
          "bbox": v["bbox"],
          "bbox_mode": BoxMode.XYWH_ABS,
          "segmentation":v["segmentation"],
          "category_id": 0,
      }
      if v["image_id"] == pre_id:
        objs.append(obj)
      else:
        filename = os.path.join("train", imgs_anns[idx-1]["file_name"])
        filename = os.path.join(data_dirs, filename)
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = pre_id
        record["height"] = height
        record["width"] = width
        record["annotations"] = objs
        dataset.append(record)

        objs = []
        record = {}
        objs.append(obj)
      
      pre_id = v["image_id"]
      idx += 1

    record["annotations"] = objs
    filename = os.path.join("train", imgs_anns[-1]["file_name"])
    filename = os.path.join(data_dirs, filename)
    height, width = cv2.imread(filename).shape[:2]
    record["file_name"] = filename
    record["image_id"] = pre_id
    record["height"] = height
    record["width"] = width
    dataset.append(record)

  return dataset

### Network

In [43]:
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("plane_train",)
cfg.DATASETS.TEST = ("plane_test",)
cfg.DATALOADER.NUM_WORKERS = 2 
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2 # the real "batch size"
cfg.SOLVER.BASE_LR = 0.001 # LR
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = [] # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 # The "RoIHead batch size"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

### Training

In [44]:
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
import copy
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_test_loader, build_detection_train_loader

def custom_mapper(dataset_dict):
  dataset_dict = copy.deepcopy(dataset_dict) # it will be modified by code below
  image = utils.read_image(dataset_dict["file_name"], format = "BGR")
  transform_list = [T.Resize((800, 800)),
                    T.RandomBrightness(0.8, 1.8),
                    T.RandomContrast(0.6, 1.3),
                    T.RandomSaturation(0.8, 1.4),
                    T.RandomRotation(angle=[90, 90]),
                    T.RandomLighting(0.7),
                    T.RandomFlip(prob=0.4, horizontal=False, vertical=True),]
  image, transforms = T.apply_transform_gens(transform_list, image)
  dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))
  annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
  instances = utils.annotations_to_instances(annos, image.shape[:2])
  dataset_dict["instances"] = utils.filter_empty_instances(instances)
  return dataset_dict

class CustomTrainer(DefaultTrainer):
  @classmethod
  def build_train_loader(cls, cfg):
     return build_detection_train_loader(cfg, mapper=custom_mapper)

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [46]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
# Visualize the output for 3 random test samples
dataset = get_detection_data("test")
for d in random.sample(dataset, 3):
  img = cv2.imread(d["file_name"])
  outputs = predictor(img)

  v = Visualizer(img[:, :, ::-1], metadata=plane_matadata, scale = 0.5)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# Qualitative comparison
dataset = get_detection_data("test")
d = dataset[1]
img = cv2.imread(d["file_name"])
outputs = predictor(img)
v = Visualizer(img[:, :, ::-1], metadata=plane_matadata, scale = 0.5)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
evaluator = COCOEvaluator("plane_train", cfg, False, output_dir= "./output")
val_loader = build_detection_test_loader(cfg, "plane_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))